# Trabalho Final - Simulated Annealing

In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
# Load the data from Excel
file = 'https://github.com/KanonStarbringer/Trabalho-Final-Modelagem-com-PL-PI/raw/main/trabalho%20final.xlsx'
df = pd.read_excel(file, sheet_name='Instancia 1')

In [3]:
# Extract data
## Extraindo os parametros do DataFrame
n = df.shape[1] - 1
r = df.iloc[0, 1:].tolist()
d = df.iloc[1, 1:].tolist()
p = df.iloc[2, 1:].tolist()
w = df.iloc[3, 1:].tolist()
tipos = df.iloc[4, 1:].tolist()

In [4]:
# Number of jobs
n = len(r)
T = max(p) + sum(d)  # Planning horizon

In [5]:
# Evaluate fitness function
def evaluate(schedule):
    total_cost = 0
    end_times = np.zeros(n)
    for i in range(n):
        job = schedule[i]
        start_time = max(r[job], end_times[i-1] if i > 0 else 0)
        end_time = start_time + d[job]
        delay = max(0, end_time - p[job])
        penalty = 0
        if delay > 0:
            if delay <= 5:
                penalty = w[job] * delay
            else:
                penalty = w[job] * 5 + w[job] * 2 * (delay - 5)
        total_cost += penalty
        end_times[i] = end_time
        
        # Machine changeover
        if i > 0 and tipos[schedule[i-1]] != tipos[job]:
            start_time += 1
            end_time += 1
            end_times[i] += 1
            
    return total_cost

In [6]:
# Generate initial solution
def generate_initial_solution():
    return list(np.random.permutation(n))

In [7]:
# Generate neighbor solution
def generate_neighbor(solution):
    neighbor = solution.copy()
    i, j = random.sample(range(n), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

In [8]:
# Simulated Annealing
def simulated_annealing(initial_temp, cooling_rate, max_iter):
    current_solution = generate_initial_solution()
    current_cost = evaluate(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temp
    
    for iteration in range(max_iter):
        neighbor = generate_neighbor(current_solution)
        neighbor_cost = evaluate(neighbor)
        
        if neighbor_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - neighbor_cost) / temperature):
            current_solution = neighbor
            current_cost = neighbor_cost
            
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
        
        temperature *= cooling_rate
        
        if temperature < 1e-10:
            break
            
    return best_solution, best_cost

In [9]:
# Run the algorithm
best_schedule, best_cost = simulated_annealing(initial_temp=1000, cooling_rate=0.99, max_iter=10000)
print(f"Best schedule: {best_schedule}")
print(f"Best cost: {best_cost}")

Best schedule: [3, 2, 5, 1, 0, 6, 4]
Best cost: 76.0
